## Анализ базы данных средствами SQL

### Задача:

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание датасетов  

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<a id='back'><a/> 
# Задачи исследования
    
### [1. Исследование таблиц](#1)
### [2. Ответы на вопросы](#1)    
#### [2.1 Сколько книг вышло после 1 января 2000 года?](#2.1)   
#### [2.2 Для каждой книги посчитайте количество обзоров и среднюю оценку](#2.2)      
#### [2.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры](#2.3)    
#### [2.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками](#2.4) 
#### [2.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#2.5)    

<a id='1'><a/> 
## 1. Исследование таблиц
[Назад к оглавлению](#back)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Таблица books

In [2]:
query = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


### Таблица authors

In [4]:
query = '''
            SELECT * 
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


### Таблица publishers

In [6]:
query = '''
            SELECT * 
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


### Таблица ratings

In [8]:
query = '''
            SELECT * 
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


### Таблица reviews

In [10]:
query = '''
            SELECT * 
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


<a id='2'><a/> 
## 2. Ответы на вопросы
[Назад к оглавлению](#back)

<a id='2.1'><a/> 
### 2.1 Сколько книг вышло после 1 января 2000 года?
[Назад к оглавлению](#back)

In [11]:
query = '''
            SELECT COUNT(book_id) count_of_books
            FROM books 
            WHERE publication_date > '01-01-2000'
        '''
pd.io.sql.read_sql(query, con = engine)

,count_of_books
0,819


#### Вывод  

После 1 января 2000 года было выпущено **819 книг**. 

<a id='2.2'><a/> 
### 2.2 Для каждой книги посчитайте количество обзоров и среднюю оценку
[Назад к оглавлению](#back)

In [12]:
query = '''
            SELECT b.book_id, rat.avg_rating, rev.review_count
            FROM books b
            LEFT JOIN (SELECT book_id, AVG(rating) avg_rating FROM ratings GROUP BY book_id) rat ON b.book_id=rat.book_id
            LEFT JOIN (SELECT book_id, COUNT(*) review_count FROM reviews GROUP BY book_id) rev ON b.book_id=rev.book_id
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,avg_rating,review_count
0,652,4.500000,2.0
1,273,4.500000,2.0
2,51,4.250000,5.0
3,951,4.000000,2.0
4,839,4.285714,4.0
...,...,...,...
995,64,4.230769,4.0
996,55,5.000000,2.0
997,148,3.428571,3.0
998,790,3.500000,2.0


#### Вывод  

Как предварително видно по данным, для большинства книг было написано **несколько обзоров** и **средние оценки варьируются**.

<a id='2.3'><a/> 
### 2.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
[Назад к оглавлению](#back)

In [13]:
query = '''
            SELECT b.publisher_id, p.publisher, COUNT(*) count_books
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY COUNT(*) DESC
            LIMIT 1
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


#### Вывод  

Издательство **"Penguin Books"** выпустило **42 книги толще 50 страниц**. Это около 4% от всех книг в данных.  

британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном. Главной заслугой издательства считают демократизацию книжного рынка, превратившую книгу из предмета роскоши в удовольствие, доступное массам. Похоже, это издательство действительно достойно внимания.

<a id='2.4'><a/> 
### 2.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
[Назад к оглавлению](#back)

In [14]:
query = '''
            SELECT a.author, AVG(ratings)
            FROM (
            SELECT book_id, AVG(rating) ratings
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(*) > 50
            ) rat
            JOIN (SELECT book_id, author_id FROM books) b ON rat.book_id=b.book_id
            JOIN (SELECT author_id, author FROM authors) a ON b.author_id=a.author_id
            GROUP BY a.author
            ORDER BY avg(ratings) DESC

            
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


#### Вывод  

А втор с самой высокой оценкой - **Джоан Роулинг**, автор книг о Гарри Пттере. Тут она представлена в соавторстве с Мэри Гранпрэ(иллюстратором). Что не удивительно, потому что книги о Гарри Поттере одни из самых популярных в мире среди всех возрастных категорий.

<a id='2.5'><a/> 
### 2.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок
[Назад к оглавлению](#back)

In [16]:
query = '''
            SELECT AVG(count_reviews)
            FROM (
            SELECT username, COUNT(*) count_reviews
            FROM reviews
            WHERE username IN
            (
            SELECT username--, COUNT(*)
            FROM ratings
            GROUP BY username
            HAVING COUNT(*) > 50
            )
            GROUP BY username) rev
        

        '''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


#### Вывод  

Среднее количество обзоров от пользователей, которые поставили более 50 оценок - **24**.   
Похоже, что люди, которые любят читать и читают много, так же любят поделиться своим мнением с другими любителями чтения :)